In [0]:
import time
start_time = time.time()


In [0]:
gold_csv_path = "/Volumes/workspace/default/sentiment_analysis/final_ml_predictions.csv"

df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(gold_csv_path)
)

df.display()
df.printSchema()


In [0]:
from pyspark.sql.functions import col, to_date, trim

gold_base_df = (
    df
    .withColumn(
        "date",
        to_date(col("created_at"), "dd-MM-yyyy HH:mm")
    )
    .withColumn("hashtags", trim(col("hashtags")))
)


In [0]:
%sql
DROP TABLE IF EXISTS default.gold_daily_hashtag;


gold_daily_hashtag

In [0]:
from pyspark.sql.functions import explode, split, count

gold_daily_hashtag = (
    gold_base_df
    .withColumn("hashtag", explode(split(col("hashtags"), ",")))
    .filter(col("hashtag").isNotNull() & (col("hashtag") != ""))
    .groupBy("date", "hashtag")
    .agg(count("*").alias("hashtag_count"))
)

gold_daily_hashtag.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.gold_daily_hashtag")


gold_daily_sentiment

In [0]:
gold_daily_sentiment = (
    gold_base_df
    .groupBy("date", col("sentiment_label").alias("sentiment"))
    .agg(count("*").alias("sentiment_count"))
)

gold_daily_sentiment.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.gold_daily_sentiment")


In [0]:
gold_hashtag_trends = (
    gold_base_df
    .withColumn("hashtag", explode(split(col("hashtags"), ",")))
    .filter(col("hashtag").isNotNull() & (col("hashtag") != ""))
    .groupBy("hashtag")
    .agg(count("*").alias("total_count"))
    .orderBy(col("total_count").desc())
)

gold_hashtag_trends.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.gold_hashtag_trends")


In [0]:
from pyspark.sql.functions import sum, round

sentiment_counts = (
    gold_base_df
    .groupBy(col("sentiment_label").alias("sentiment"))
    .agg(count("*").alias("count"))
)

total_count = sentiment_counts.agg(sum("count")).collect()[0][0]

gold_sentiment_percentage = (
    sentiment_counts
    .withColumn(
        "percentage",
        round((col("count") / total_count) * 100, 2)
    )
)

gold_sentiment_percentage.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.gold_sentiment_percentage")


In [0]:
from pyspark.sql.functions import sum as _sum

gold_top_users = (
    gold_base_df
    .groupBy("username", "user_id")
    .agg(
        _sum("retweet_count").alias("total_retweets"),
        _sum("like_count").alias("total_likes"),
        _sum("reply_count").alias("total_replies")
    )
    .withColumn(
        "engagement_score",
        col("total_retweets") +
        col("total_likes") +
        col("total_replies")
    )
    .orderBy(col("engagement_score").desc())
)

gold_top_users.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.gold_top_users")


In [0]:
from pyspark.sql.functions import (
    col, explode, split, count, trim,
    to_timestamp, to_date
)

gold_df = (
    gold_base_df
    .withColumn("hashtag", explode(split(col("hashtags"), ",")))
    .withColumn("hashtag", trim(col("hashtag")))
    .filter(col("hashtag").isNotNull() & (col("hashtag") != ""))
    
    .withColumn(
        "created_ts",
        to_timestamp(col("created_at"), "dd-MM-yyyy HH:mm")
    )
    .withColumn("date", to_date(col("created_ts")))
    .groupBy("date", "hashtag")
    .agg(count("*").alias("count"))
)

gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_trending_topics")


In [0]:
end_time = time.time()
print(f"Total notebook runtime: {(end_time - start_time)/60:.2f} minutes")
